# Analysis of StackOverflow Survey. Part IV 

In this notebook we build a predictiv model for job satisfaction. 

In [5]:
# import neccessary packages and libraries
import os
from collections import defaultdict

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# to render plots in the notebook
%matplotlib inline

import seaborn as sns
# set a theme for seaborn
sns.set_theme()

from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn import (
    ensemble,
    preprocessing,
    tree,
)
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
)
from sklearn.metrics import (
    classification_report,
    r2_score, 
    mean_squared_error,
    auc,
    confusion_matrix,
    accuracy_score,
    roc_auc_score,
    roc_curve,
)


In [2]:
# import local module containing the neccessary functions
import utils_functions as uf

# forces the interpreter to re-import the module
import importlib
importlib.reload(uf);

## State the question
I am addressing the third question in this notebook. What can we tell about the job satisfaction of a data coder? What factors do influence it? Also, predict the job satisfaction for a developer who works with big data. 

This is a classification question, we are predicting a satisfaction level for a data developer, which includes: data scientist or machine learning specialist, data or business analyst and data engineer.

## Performance metrics - to review at the end

The following performance measures will be used in this project:
1. Cross validation via StratifiedKFold with 10 folds.
2. Confusion matrix, in particular precision, recall and F1 score.
3. The ROC curve and the related AUC score.

## Gather the data

Upload the data and keep the subset that contains those developers that work in data science related fields.

In [47]:
# create a path string
mypath = os.getcwd()

# upload the datafiles as pandas dataframes
df1 = pd.read_csv(mypath+'/data/survey20_updated.csv')

# check the uploaded data
df1.shape

(64461, 25)

In [48]:
# the data frame that contains the data developers only
df1 = df1[df1.DevClass == 'data_coder']

# check the size of the data
df1.shape

(8726, 25)

In [49]:
# create a list of columns to be used in this analysis
list_cols = ['MainBranch', 'ConvertedComp', 
       'EdLevel', 'Employment',
       'JobSat', 'EdImpt',
       'Learn', 'Overtime', 'OpSys', 'OrgSize', 
       'UndergradMajor', 'WorkWeekHrs']

In [50]:
# the dataset that contains only the listed columns
df1 = df1[list_cols]
df1.shape

(8726, 12)

In [51]:
# reset the index 
df1.reset_index(drop=True, inplace=True)

In [52]:
# gather information on dtypes and missing values
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8726 entries, 0 to 8725
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MainBranch      8699 non-null   object 
 1   ConvertedComp   5810 non-null   float64
 2   EdLevel         8581 non-null   object 
 3   Employment      8726 non-null   object 
 4   JobSat          8726 non-null   int64  
 5   EdImpt          8206 non-null   object 
 6   Learn           7979 non-null   object 
 7   Overtime        7424 non-null   object 
 8   OpSys           8120 non-null   object 
 9   OrgSize         7590 non-null   object 
 10  UndergradMajor  8053 non-null   object 
 11  WorkWeekHrs     6995 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 818.2+ KB


## Data profiling

In [53]:
# run this once to generate a profiling report and save it as html file

#import pandas_profiling
#profile = pandas_profiling.ProfileReport(df, minimal=False)
#profile.to_file(output_file="data_coders_report.html")

## Data preprocessing 

### Remove duplicates

In [54]:
# drop duplicate rows, if any
df1.drop_duplicates(subset=None, keep='first', inplace=True)
df1.shape

(8553, 12)

### Create bins for the WorkWeekHrs column

In [55]:
# create the labels
cut_labels = ['<10', '10-20', '20-30', '30-40', '40-50', '>50']

# define the bins 
m = df1.WorkWeekHrs.max()
cut_bins = [0, 10, 20, 30, 40, 50, m]

# create a new column which contains the new labels
df1['WorkWeek_Bins'] = pd.cut(df1['WorkWeekHrs'], bins=cut_bins, labels=cut_labels)

# check for success
df1['WorkWeek_Bins'].value_counts()

30-40    3842
40-50    1836
>50       610
<10       284
20-30     276
10-20     140
Name: WorkWeek_Bins, dtype: int64

In [56]:
# count the missing values in the new column
df1['WorkWeek_Bins'].isnull().sum()

1565

In [57]:
# change the type of the newly created column
df1['WorkWeek_Bins'] = df1['WorkWeek_Bins'].astype('object')

In [58]:
# drop the WorkWeekHrs column
df1.drop(columns = 'WorkWeekHrs', inplace=True);

### Create bins for the ConvertedComp column

In [59]:
# we could use quantile, however I prefer custom bins here
cut_labels = ['<10K', '10K-30K', '30K-50K', '50K-100K', '100K-200K', '>200K']

# define the bins 
m = df1.ConvertedComp.max()
cut_bins = [0, 10000, 30000, 50000, 100000, 200000, m]

# create a new column which contains the new labels
df1['Comp_Bins'] = pd.cut(df1['ConvertedComp'], bins=cut_bins, labels=cut_labels)

# change the type of the newly created column
df1['Comp_Bins'] = df1['Comp_Bins'].astype('object')

# drop the WorkWeekHrs column
df1.drop(columns = 'ConvertedComp', inplace=True);

## Create features and target

Create a dataframe (X) with the features and a pandas series (y) that contains the labels.

In [64]:
# create a copy of the pre-processed dataframe
df2 = df1.copy()

In [65]:
# create the predictors dataframe
X = df2.drop(columns = 'JobSat')

# create the labels
y = df2['JobSat']

# check for success
X.info(), len(y)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8553 entries, 0 to 8725
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   MainBranch      8526 non-null   object
 1   EdLevel         8410 non-null   object
 2   Employment      8553 non-null   object
 3   EdImpt          8144 non-null   object
 4   Learn           7818 non-null   object
 5   Overtime        7417 non-null   object
 6   OpSys           7954 non-null   object
 7   OrgSize         7580 non-null   object
 8   UndergradMajor  7905 non-null   object
 9   WorkWeek_Bins   6988 non-null   object
 10  Comp_Bins       5783 non-null   object
dtypes: object(11)
memory usage: 801.8+ KB


(None, 8553)

## Create dummies for the dataframe of predictors X

In [75]:
# create dummies for all the columns in dataframe
X_dumm = pd.get_dummies(X)

# check for success
X_dumm.shape

(8553, 69)

## Sample data

We will use $30 \%$ data for testing:

In [79]:
# split the data in train and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_dumm, y, test_size=0.3, random_state=42)

# check for success
X_train.shape, len(y_train), X_test.shape, len(y_test)

((5987, 69), 5987, (2566, 69), 2566)

## Impute missing values

Now that we have test and train data, we can impute missing values on the training set, and use the trained imputer to fill in the test dataset. I will use the KNN imputer from sklearn.

In [85]:
# create an instance of the imputer
imputer = KNNImputer(n_neighbors=5)

# fit the imputer on the dataset
X_train_trans = pd.DataFrame(imputer.fit_transform(X_train), columns = X_train.columns)

# check for success
X_train_trans.isna().any()

MainBranch_I am a developer by profession                                                   False
MainBranch_I am a student who is learning to code                                           False
MainBranch_I am not primarily a developer, but I write code sometimes as part of my work    False
MainBranch_I code primarily as a hobby                                                      False
MainBranch_I used to be a developer by profession, but no longer am                         False
                                                                                            ...  
Comp_Bins_10K-30K                                                                           False
Comp_Bins_30K-50K                                                                           False
Comp_Bins_50K-100K                                                                          False
Comp_Bins_<10K                                                                              False
Comp_Bins_>200K     

## Refactor code


In [4]:
# create a path string
mypath = os.getcwd()

# read the data from the file
df = pd.read_csv(mypath+'/data/survey20_updated.csv')
# preprocess, split and process data
preproc_df = uf.preprocess_data(df)
X_train, y_train, X_test, y_test = uf.process_data(preproc_df, 'JobSat')

## Baseline model: K NearestNeighbors

In [16]:
# create an instance of the classifier
knn_clf = KNeighborsClassifier(n_neighbors=10)

# fit the classifier
knn_clf.fit(X_train, y_train)

# predict output values
y_pred = knn_clf.predict(X_test)

In [17]:
# print evaluation metrics and results

result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)
         

Confusion Matrix:
[[263   1   1   2   4   8]
 [  5   9  12   8  81  67]
 [ 11  14  47  21 144 119]
 [ 10  10  29  18 116 104]
 [ 11  13  75  34 287 249]
 [ 33  14  70  40 285 351]]

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.94      0.86       279
           1       0.15      0.05      0.07       182
           2       0.20      0.13      0.16       356
           3       0.15      0.06      0.09       287
           4       0.31      0.43      0.36       669
           5       0.39      0.44      0.42       793

    accuracy                           0.38      2566
   macro avg       0.33      0.34      0.33      2566
weighted avg       0.34      0.38      0.35      2566

Accuracy: 0.380


## Several other algorithms 

In [19]:
from sklearn.svm import SVC

# create classifier instance
svm_clf = SVC(gamma="auto", random_state=42)
# fit the model
svm_clf.fit(X_train, y_train)

SVC(gamma='auto', random_state=42)

In [52]:
# predict on the test set
y_pred = svm_clf.predict(X_test)

# test one value
y_test.iloc[20],  y_pred[20]

(5, 4)

In [55]:
some_digit_scores = svm_clf.decision_function(X_test)
some_digit_scores[20]

array([-0.28494909,  0.74793612,  3.16192468,  1.90535375,  5.2713231 ,
        4.2675518 ])

In [56]:
np.argmax(some_digit_scores[20])

4

In [57]:
svm_clf.classes_

array([0, 1, 2, 3, 4, 5])

In [41]:
y_test.values[0]

2

In [59]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm_clf, X_train, y_train, cv=3, scoring="accuracy")

array([0.39378758, 0.42084168, 0.41002506])

In [63]:
# print evaluation metrics and results

result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred, zero_division=0)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)
         

Confusion Matrix:
[[251   0   0   0  12  16]
 [  0   0   0   0  62 120]
 [  0   0   0   0 141 215]
 [  0   0   0   0 109 178]
 [  0   0   0   0 252 417]
 [  0   0   0   0 181 612]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95       279
           1       0.00      0.00      0.00       182
           2       0.00      0.00      0.00       356
           3       0.00      0.00      0.00       287
           4       0.33      0.38      0.35       669
           5       0.39      0.77      0.52       793

    accuracy                           0.43      2566
   macro avg       0.29      0.34      0.30      2566
weighted avg       0.32      0.43      0.36      2566

Accuracy: 0.435


In [64]:
from sklearn.multiclass import OneVsRestClassifier
ovr_clf = OneVsRestClassifier(SVC(gamma="auto", random_state=42))
ovr_clf.fit(X_train, y_train)
y_pred = ovr_clf.predict(X_test)

In [65]:
result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred, zero_division=0)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)

Confusion Matrix:
[[272   2   1   1   0   3]
 [  5   5  15   9  43 105]
 [ 11  12  31  15 112 175]
 [ 15  10  29  16  85 132]
 [ 18  26  42  37 181 365]
 [ 32  33  44  28 174 482]]

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.97      0.86       279
           1       0.06      0.03      0.04       182
           2       0.19      0.09      0.12       356
           3       0.15      0.06      0.08       287
           4       0.30      0.27      0.29       669
           5       0.38      0.61      0.47       793

    accuracy                           0.38      2566
   macro avg       0.31      0.34      0.31      2566
weighted avg       0.33      0.38      0.34      2566

Accuracy: 0.385


In [72]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Always scale the input. The most convenient way is to use a pipeline.
sgd_clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
sgd_clf.fit(X_train, y_train)
y_pred = sgd_clf.predict(X_test)

In [73]:
result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred, zero_division=0)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)

Confusion Matrix:
[[265   2   3   3   6   0]
 [  0  17  17  19  56  73]
 [  1  19  46  27 135 128]
 [  0  23  32  27 101 104]
 [  1  54  67  66 239 242]
 [  4  47  70  79 256 337]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.95      0.96       279
           1       0.10      0.09      0.10       182
           2       0.20      0.13      0.16       356
           3       0.12      0.09      0.11       287
           4       0.30      0.36      0.33       669
           5       0.38      0.42      0.40       793

    accuracy                           0.36      2566
   macro avg       0.35      0.34      0.34      2566
weighted avg       0.35      0.36      0.35      2566

Accuracy: 0.363


In [66]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")

NameError: name 'sgd_clf' is not defined

In [74]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [76]:
from sklearn import model_selection
from sklearn.linear_model import (LogisticRegression)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import (KNeighborsClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import (RandomForestClassifier)
import xgboost as xgb
from sklearn.metrics import classification_report

In [78]:
for model in [DecisionTreeClassifier, KNeighborsClassifier, GaussianNB, SVC, 
              RandomForestClassifier, xgb.XGBClassifier]:
    cls = model()
    kfold = model_selection.KFold(n_splits=10)
    #result2 = classification_report(y_test, y_pred, zero_division=0)
    s = model_selection.cross_val_score(cls, X, y, cv=kfold)
    #print(f"{model.__name__:22}  AUC:" f"{s.mean():.3f} STD: {s.std():.2f}")

/home/silvia/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/silvia/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/silvia/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py", line 726, in fit
    missing=self.missing, nthread=self.n_jobs)
  File "/home/silvia/anaconda3/lib/python3.7/site-packages/xgboost/core.py", line 426, in __init__
    self.feature_names = feature_names
  File "/home/silvia/anaconda3/lib/python3.7/site-packages/xgboost/core.py", line 870, in feature_names
    raise ValueError('feature_names may not contain [, ] or <')
ValueError: feature_names may not contain [, ] or <

  FitFailedWarning)
/home/silvia/anaconda3/lib/python3.7/si

In [ ]:
print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
cls = xgb.XGBClassifier()
kfold = model_selection.KFold(n_splits=10)
s = model_selection.cross_val_score(cls, X, y, scoring="roc_auc", cv=kfold)
    print(f"{model.__name__:22}  AUC:" f"{s.mean():.3f} STD: {s.std():.2f}")

In [ ]:
diabetes = load_diabetes()

X = diabetes.data
y = diabetes.target

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(X):   
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    xgb_model = xgb.XGBRegressor(objective="reg:linear")
    xgb_model.fit(X_train, y_train)
    
    y_pred = xgb_model.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))
    
display_scores(np.sqrt(scores))